In [1]:
from AlgoTools import AlgoTools
from Trader import Trader
import pandas as pd
import time
import datetime
import pickle
import numpy as np

# Processing the data
- The data has already been grouped into days and converted to date time objects
- Objective is to shuffle the days, then create a train-test split as I will be training on indivuals days

In [2]:
with open('SPY_Featured.pickle', 'rb') as f:
    spDaily = pickle.load(f)

In [3]:
spDaily.head()

,price,DateTime,velocity,acceleration,Direction
4,1495.5,2000-01-03 08:30:53,0.071429,0.026871,0
7,1495.0,2000-01-03 08:31:04,-0.045455,-0.010626,0
9,1494.7,2000-01-03 08:31:12,-0.037500,0.000994,0
11,1494.5,2000-01-03 08:31:19,-0.028571,0.001276,1
14,1495.0,2000-01-03 08:31:26,0.071429,0.014286,1


In [4]:
spDaily['percent_increase'] = (spDaily['price'].pct_change() * 100)


In [7]:
spDaily

,price,DateTime,velocity,acceleration,Direction,percent_increase
7,1495.0,2000-01-03 08:31:04.000,-0.045455,-1.062574e-02,0,-0.033434
9,1494.7,2000-01-03 08:31:12.000,-0.037500,9.943182e-04,0,-0.020067
11,1494.5,2000-01-03 08:31:19.000,-0.028571,1.275510e-03,1,-0.013381
14,1495.0,2000-01-03 08:31:26.000,0.071429,1.428571e-02,1,0.033456
15,1495.3,2000-01-03 08:31:46.000,0.015000,-2.821429e-03,0,0.020067
...,...,...,...,...,...,...
13802495,3218.0,2019-12-31 12:22:01.231,0.000144,2.066065e-08,1,0.062189
13802496,3227.5,2019-12-31 14:48:18.941,0.001082,1.069096e-07,1,0.295214
13802498,3231.5,2019-12-31 15:00:09.277,0.005631,6.403802e-06,1,0.123935
13802500,3233.0,2019-12-31 15:01:09.088,0.025079,3.251553e-04,0,0.046418


In [11]:
X=spDaily[["velocity","acceleration","percent_increase"]]

In [13]:
y=spDaily["Direction"]

In [15]:
X

,velocity,acceleration,percent_increase
7,-0.045455,-1.062574e-02,-0.033434
9,-0.037500,9.943182e-04,-0.020067
11,-0.028571,1.275510e-03,-0.013381
14,0.071429,1.428571e-02,0.033456
15,0.015000,-2.821429e-03,0.020067
...,...,...,...
13802495,0.000144,2.066065e-08,0.062189
13802496,0.001082,1.069096e-07,0.295214
13802498,0.005631,6.403802e-06,0.123935
13802500,0.025079,3.251553e-04,0.046418


In [16]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X.iloc[:100000], y.iloc[:100000], test_size=0.2, random_state=42)


In [21]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [22]:
model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])

In [25]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [26]:
history = model.fit(X_train, y_train, epochs=50, batch_size=8, validation_split=0.2)


Epoch 1/50
   1/8000 [..............................] - ETA: 18:36 - loss: 0.9071 - accuracy: 0.2500

2024-11-09 14:32:36.841285: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


8000/8000 [==============================] - 3s 304us/step - loss: 0.6803 - accuracy: 0.5656 - val_loss: 0.6795 - val_accuracy: 0.5676
Epoch 2/50
8000/8000 [==============================] - 2s 285us/step - loss: 0.6794 - accuracy: 0.5673 - val_loss: 0.6798 - val_accuracy: 0.5681
Epoch 3/50
8000/8000 [==============================] - 2s 282us/step - loss: 0.6792 - accuracy: 0.5666 - val_loss: 0.6790 - val_accuracy: 0.5683
Epoch 4/50
8000/8000 [==============================] - 2s 285us/step - loss: 0.6791 - accuracy: 0.5663 - val_loss: 0.6794 - val_accuracy: 0.5687
Epoch 5/50
8000/8000 [==============================] - 2s 284us/step - loss: 0.6790 - accuracy: 0.5671 - val_loss: 0.6792 - val_accuracy: 0.5681
Epoch 6/50
8000/8000 [==============================] - 2s 283us/step - loss: 0.6790 - accuracy: 0.5670 - val_loss: 0.6794 - val_accuracy: 0.5691
Epoch 7/50
8000/8000 [==============================] - 2s 285us/step - loss: 0.6789 - accuracy: 0.5678 - val_loss: 0.6790 - val_accura

In [27]:

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy:.4f}')


625/625 [==============================] - 0s 234us/step - loss: 0.6797 - accuracy: 0.5667
Test Accuracy: 0.5667


In [43]:
class_counts = y_test.value_counts()

# Display the counts
print(f'Count of 0s: {class_counts[0]}')
print(f'Count of 1s: {class_counts[1]}')

Count of 0s: 11297
Count of 1s: 8703


In [44]:
11297/20000

0.56485

In [29]:
import random

random_number = random.randint(1, len(X))

print(random_number)

4153090


In [28]:
len(X)

4541647

In [31]:
print(spDaily.iloc[4153092,:])

price                                   1321.7
DateTime            2012-05-30 07:35:43.780000
velocity                             -0.004088
acceleration                         -0.000009
Direction                                    1
percent_increase                     -0.136003
Name: 12780769, dtype: object


In [37]:
new_data_scaled = scaler.transform([X.iloc[4153092,:]])

# Make a prediction
prediction = model.predict(new_data_scaled)

# Convert prediction to binary output (0 or 1) if needed
predicted_class = (prediction > 0.5).astype("int32")  

1/1 [==============================] - 0s 13ms/step


/Users/atulkrishnan/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [38]:
predicted_class

array([[0]], dtype=int32)

In [52]:
Prediction = model.predict(X_test) 

625/625 [==============================] - 0s 202us/step


In [55]:
Prediction = Prediction.flatten()

In [67]:
Prediction = (Prediction> 0.5).astype("int32")

In [69]:
counts = np.bincount(Prediction)

print(f'Count of 0s: {counts[0]}')
print(f'Count of 1s: {counts[1]}')

Count of 0s: 17370
Count of 1s: 2630


In [68]:
import plotly.graph_objects as go

# Example list of numbers

# Create the plot
fig = go.Figure(data=go.Scatter(y=Prediction, mode='markers'))

# Customize the layout
fig.update_layout(
    title='Plot of Numbers',
    xaxis_title='Index',
    yaxis_title='Value',
)

# Show the plot
fig.show()